In [4]:
# ict-irt mapping and logging app.

import ipywidgets as widgets
from ipyleaflet import Map, Marker, Icon
from IPython.display import display, clear_output
import base64
import pandas as pd
from datetime import datetime, timedelta

# ---- Data + SVGs (abbreviated for brevity) ----
symbol_options = [
    "Friendly Infantry", "Hostile Infantry", "Neutral Infantry", "Unknown Infantry",
    "Friendly Armor", "Hostile Armor", "Friendly Artillery", "Hostile Artillery",
    "Friendly Engineer", "Friendly Signal", "Friendly Medical",
    "Friendly Recon", "Friendly SF", "Friendly HQ",
    "Friendly CBRN", "Friendly UAV", "Friendly Radar",
    "Friendly INT", "Hostile INT", "Neutral INT", "Unknown INT",
    "Friendly SIGINT", "Friendly ELINT",
    "Friendly Air", "Hostile Air", "Neutral Air",
    "Friendly Naval", "Hostile Naval", "Neutral Naval"
]

SVGs = {
    # --- Land Forces with silhouettes ---
    "Friendly Infantry": """<svg xmlns="http://www.w3.org/2000/svg" width="80" height="60">
      <rect width="80" height="60" fill="blue"/>
      <!-- Crossed rifles style -->
      <g stroke="white" stroke-width="4" stroke-linecap="round">
        <line x1="25" y1="45" x2="55" y2="15"/>
        <line x1="55" y1="45" x2="25" y2="15"/>
      </g>
    </svg>""",
    "Hostile Infantry": """<svg xmlns="http://www.w3.org/2000/svg" width="80" height="60">
      <polygon points="40,4 76,30 40,56 4,30" fill="red"/>
      <g stroke="white" stroke-width="4" stroke-linecap="round">
        <line x1="25" y1="45" x2="55" y2="15"/>
        <line x1="55" y1="45" x2="25" y2="15"/>
      </g>
    </svg>""",
    "Neutral Infantry": """<svg xmlns="http://www.w3.org/2000/svg" width="80" height="60">
      <rect width="80" height="60" fill="green"/>
      <g stroke="white" stroke-width="4" stroke-linecap="round">
        <line x1="25" y1="45" x2="55" y2="15"/>
        <line x1="55" y1="45" x2="25" y2="15"/>
      </g>
    </svg>""",
    "Unknown Infantry": """<svg xmlns="http://www.w3.org/2000/svg" width="80" height="60">
      <ellipse cx="40" cy="30" rx="38" ry="28" fill="yellow"/>
      <g stroke="black" stroke-width="4" stroke-linecap="round">
        <line x1="25" y1="45" x2="55" y2="15"/>
        <line x1="55" y1="45" x2="25" y2="15"/>
      </g>
    </svg>""",
    "Friendly Armor": """<svg xmlns="http://www.w3.org/2000/svg" width="80" height="60">
      <rect width="80" height="60" fill="blue"/>
      <!-- Stylized tank tread -->
      <rect x="22" y="32" width="36" height="10" fill="white"/>
      <circle cx="30" cy="37" r="4" fill="blue"/>
      <circle cx="50" cy="37" r="4" fill="blue"/>
      <rect x="32" y="22" width="16" height="10" fill="white"/>
    </svg>""",
    "Hostile Armor": """<svg xmlns="http://www.w3.org/2000/svg" width="80" height="60">
      <polygon points="40,4 76,30 40,56 4,30" fill="red"/>
      <rect x="22" y="32" width="36" height="10" fill="white"/>
      <circle cx="30" cy="37" r="4" fill="red"/>
      <circle cx="50" cy="37" r="4" fill="red"/>
      <rect x="32" y="22" width="16" height="10" fill="white"/>
    </svg>""",

    "Friendly Artillery": """<svg xmlns="http://www.w3.org/2000/svg" width="80" height="60">
      <rect width="80" height="60" fill="blue"/>
      <!-- Artillery cannon (barrel and wheel) -->
      <circle cx="40" cy="38" r="8" fill="white"/>
      <rect x="37" y="18" width="6" height="20" fill="white"/>
      <rect x="34" y="14" width="12" height="6" fill="white"/>
    </svg>""",
    "Hostile Artillery": """<svg xmlns="http://www.w3.org/2000/svg" width="80" height="60">
      <polygon points="40,4 76,30 40,56 4,30" fill="red"/>
      <circle cx="40" cy="38" r="8" fill="white"/>
      <rect x="37" y="18" width="6" height="20" fill="white"/>
      <rect x="34" y="14" width="12" height="6" fill="white"/>
    </svg>""",
    "Friendly Engineer": """<svg xmlns="http://www.w3.org/2000/svg" width="80" height="60">
      <rect width="80" height="60" fill="blue"/>
      <!-- Bridge shape (engineering) -->
      <rect x="26" y="34" width="28" height="8" fill="white"/>
      <polygon points="26,34 40,20 54,34" fill="white"/>
    </svg>""",
    "Friendly Signal": """<svg xmlns="http://www.w3.org/2000/svg" width="80" height="60">
      <rect width="80" height="60" fill="blue"/>
      <!-- Lightning bolt -->
      <polyline points="38,20 44,32 36,32 42,44" fill="none" stroke="white" stroke-width="4"/>
    </svg>""",
    "Friendly Medical": """<svg xmlns="http://www.w3.org/2000/svg" width="80" height="60">
      <rect width="80" height="60" fill="blue"/>
      <!-- Caduceus (medical): stylized cross -->
      <rect x="36" y="20" width="8" height="20" fill="white"/>
      <rect x="30" y="28" width="20" height="8" fill="white"/>
    </svg>""",
    "Friendly Recon": """<svg xmlns="http://www.w3.org/2000/svg" width="80" height="60">
      <rect width="80" height="60" fill="blue"/>
      <!-- Binoculars (recon) -->
      <ellipse cx="34" cy="32" rx="6" ry="10" fill="white"/>
      <ellipse cx="46" cy="32" rx="6" ry="10" fill="white"/>
      <rect x="34" y="24" width="12" height="8" fill="blue"/>
    </svg>""",
    "Friendly SF": """<svg xmlns="http://www.w3.org/2000/svg" width="80" height="60">
      <rect width="80" height="60" fill="blue"/>
      <!-- Stylized spearhead -->
      <polygon points="40,12 47,48 40,40 33,48" fill="white"/>
    </svg>""",
    "Friendly HQ": """<svg xmlns="http://www.w3.org/2000/svg" width="80" height="60">
      <rect width="80" height="60" fill="blue"/>
      <!-- HQ staff (flag at top left) -->
      <rect x="12" y="6" width="16" height="8" fill="white"/>
      <polygon points="12,6 12,14 20,10" fill="white"/>
    </svg>""",
    "Friendly CBRN": """<svg xmlns="http://www.w3.org/2000/svg" width="80" height="60">
      <rect width="80" height="60" fill="blue"/>
      <!-- Trefoil symbol (CBRN) -->
      <g fill="yellow" stroke="black" stroke-width="1">
        <circle cx="40" cy="30" r="5"/>
        <circle cx="40" cy="20" r="6"/>
        <circle cx="32" cy="36" r="6"/>
        <circle cx="48" cy="36" r="6"/>
      </g>
    </svg>""",
    "Friendly UAV": """<svg xmlns="http://www.w3.org/2000/svg" width="80" height="60">
      <rect width="80" height="60" fill="blue"/>
      <!-- Stylized drone (UAV) -->
      <rect x="36" y="24" width="8" height="16" fill="white"/>
      <line x1="20" y1="32" x2="60" y2="32" stroke="white" stroke-width="4"/>
    </svg>""",
    "Friendly Radar": """<svg xmlns="http://www.w3.org/2000/svg" width="80" height="60">
      <rect width="80" height="60" fill="blue"/>
      <!-- Parabolic radar dish -->
      <ellipse cx="40" cy="40" rx="14" ry="8" fill="white"/>
      <line x1="40" y1="40" x2="40" y2="25" stroke="white" stroke-width="3"/>
      <circle cx="40" cy="25" r="3" fill="white"/>
    </svg>""",
    # --- Intelligence / Info (use text as before) ---
    "Friendly INT": """<svg xmlns="http://www.w3.org/2000/svg" width="80" height="60">
      <rect width="80" height="60" fill="blue"/>
      <text x="40" y="35" font-size="28" fill="white" text-anchor="middle">INT</text>
    </svg>""",
    "Hostile INT": """<svg xmlns="http://www.w3.org/2000/svg" width="80" height="60">
      <polygon points="40,4 76,30 40,56 4,30" fill="red"/>
      <text x="40" y="35" font-size="28" fill="white" text-anchor="middle">INT</text>
    </svg>""",
    "Neutral INT": """<svg xmlns="http://www.w3.org/2000/svg" width="80" height="60">
      <rect width="80" height="60" fill="green"/>
      <text x="40" y="35" font-size="28" fill="white" text-anchor="middle">INT</text>
    </svg>""",
    "Unknown INT": """<svg xmlns="http://www.w3.org/2000/svg" width="80" height="60">
      <ellipse cx="40" cy="30" rx="38" ry="28" fill="yellow"/>
      <text x="40" y="35" font-size="28" fill="black" text-anchor="middle">INT</text>
    </svg>""",
    "Friendly SIGINT": """<svg xmlns="http://www.w3.org/2000/svg" width="80" height="60">
      <rect width="80" height="60" fill="blue"/>
      <text x="40" y="28" font-size="18" fill="white" text-anchor="middle">SIG</text>
      <text x="40" y="45" font-size="18" fill="white" text-anchor="middle">INT</text>
    </svg>""",
    "Friendly ELINT": """<svg xmlns="http://www.w3.org/2000/svg" width="80" height="60">
      <rect width="80" height="60" fill="blue"/>
      <text x="40" y="28" font-size="18" fill="white" text-anchor="middle">EL</text>
      <text x="40" y="45" font-size="18" fill="white" text-anchor="middle">INT</text>
    </svg>""",
    # --- Air / Naval ---
    "Friendly Air": """<svg xmlns="http://www.w3.org/2000/svg" width="80" height="60">
      <ellipse cx="40" cy="30" rx="38" ry="28" fill="blue"/>
      <!-- Chevron/triangle for air -->
      <polygon points="40,14 54,46 26,46" fill="white"/>
    </svg>""",
    "Hostile Air": """<svg xmlns="http://www.w3.org/2000/svg" width="80" height="60">
      <ellipse cx="40" cy="30" rx="38" ry="28" fill="red"/>
      <polygon points="40,14 54,46 26,46" fill="white"/>
    </svg>""",
    "Neutral Air": """<svg xmlns="http://www.w3.org/2000/svg" width="80" height="60">
      <ellipse cx="40" cy="30" rx="38" ry="28" fill="green"/>
      <polygon points="40,14 54,46 26,46" fill="white"/>
    </svg>""",
    "Friendly Naval": """<svg xmlns="http://www.w3.org/2000/svg" width="80" height="60">
      <!-- Blue horizontal oval for naval -->
      <ellipse cx="40" cy="30" rx="30" ry="12" fill="blue"/>
      <!-- Superstructure -->
      <rect x="34" y="16" width="12" height="8" fill="white"/>
    </svg>""",
    "Hostile Naval": """<svg xmlns="http://www.w3.org/2000/svg" width="80" height="60">
      <ellipse cx="40" cy="30" rx="30" ry="12" fill="red"/>
      <rect x="34" y="16" width="12" height="8" fill="white"/>
    </svg>""",
    "Neutral Naval": """<svg xmlns="http://www.w3.org/2000/svg" width="80" height="60">
      <ellipse cx="40" cy="30" rx="30" ry="12" fill="green"/>
      <rect x="34" y="16" width="12" height="8" fill="white"/>
    </svg>""",
}

def svg_to_datauri(svg):
    svg_b64 = base64.b64encode(svg.encode('utf-8')).decode('utf-8')
    return f'data:image/svg+xml;base64,{svg_b64}'
def now_str(): return datetime.now().strftime("%Y-%m-%d:%H:%M:%S")
def dt_from_str(s): return datetime.strptime(s, "%Y-%m-%d:%H:%M:%S")
def icon_size_for_zoom(zoom):
    ZOOM_ICON_TABLE = {7:[12,9],8:[16,12],9:[20,15],10:[26,19],11:[32,24],12:[40,30],13:[48,36],14:[32,24],15:[24,18],16:[16,12],17:[10,8],18:[7,5]}
    z = int(round(zoom))
    sizes = sorted(ZOOM_ICON_TABLE.items())
    for k, sz in reversed(sizes):
        if z >= k: return sz
    return sizes[0][1]

# ---- DataFrame Schema (all columns) ----
log_columns = [
    "symbol", "when", "coords", "reported_at",
    "reporter_id", "reporter_type", "source_reliability", "info_confidence",
    "description", "source_reference"
]
log_df = pd.DataFrame(columns=log_columns)

# --- UI State ---
sort_col, sort_desc = 'reported_at', True
date_filter_mode, date_filter_col = 'all', 'when'
date_filter_from, date_filter_to = '', ''
page_size, page_num = 10, 0
selected_symbols = set(symbol_options)
editing_row_idx = [None]
creating_row = [False]

edit_mode = widgets.ToggleButton(
    value=False, description="Edit Table", icon="edit", button_style="info",
    layout=widgets.Layout(width="120px", min_height="38px", max_height="38px", margin="0 0 0 14px")
)

symbol_dropdown = widgets.Dropdown(options=symbol_options, value="Friendly Infantry", description="Symbol:")
add_info = widgets.HTML(value="<b>Click map to add marker</b>", layout=widgets.Layout(margin="0 0 10px 0"))

# ---- Symbol Multi-Select for Filtering ----
symbol_select_dropdown_btn = widgets.ToggleButton(value=False, icon="filter", description="Symbols", layout=widgets.Layout(width="120px", height="32px", min_width="120px"))
symbol_select_box = widgets.SelectMultiple(options=symbol_options, value=tuple(symbol_options), rows=10, layout=widgets.Layout(width="210px", margin="2px 0 0 0", max_height="220px"))
symbol_dropdown_outer = widgets.Box(layout=widgets.Layout(position='relative', min_width="120px", max_width="140px"))
symbol_dropdown_inner = widgets.VBox([symbol_select_dropdown_btn], layout=widgets.Layout(min_width="120px", max_width="140px"))
symbol_dropdown_outer.children = [symbol_dropdown_inner]
def update_symbol_dropdown_visibility(*a):
    if symbol_select_dropdown_btn.value:
        symbol_dropdown_inner.children = [symbol_select_dropdown_btn, symbol_select_box]
    else:
        symbol_dropdown_inner.children = [symbol_select_dropdown_btn]
symbol_select_dropdown_btn.observe(update_symbol_dropdown_visibility, names='value')
def on_symbol_filter_change(change):
    global selected_symbols, page_num
    selected_symbols = set(change['new'])
    page_num = 0
    refresh_all(refresh_map_flag=False)
symbol_select_box.observe(on_symbol_filter_change, names='value')

# ---- Date Filtering Controls ----
date_filter_dropdown = widgets.Dropdown(
    options=[('All', 'all'), ('Last 1h', '1h'), ('Last 24h', '24h'), ('Custom', 'custom')],
    value='all', description='Date:', layout=widgets.Layout(width="165px", min_width="120px", max_width="180px", border="1px solid #444"))
date_col_dropdown = widgets.Dropdown(
    options=[('When', 'when'), ('Reported At', 'reported_at')],
    value='when', description='On:', layout=widgets.Layout(width="165px", min_width="120px", max_width="180px", border="1px solid #444"))
date_from_picker = widgets.Text(
    value='', placeholder='YYYY-MM-DD:HH:MM:SS', description='From:', layout=widgets.Layout(width="160px")
)
date_to_picker = widgets.Text(
    value='', placeholder='YYYY-MM-DD:HH:MM:SS', description='To:', layout=widgets.Layout(width="160px")
)
def on_date_filter_change(change):
    global date_filter_mode, page_num
    date_filter_mode = change['new']
    page_num = 0
    refresh_all(refresh_map_flag=False)
date_filter_dropdown.observe(on_date_filter_change, names='value')
def on_date_col_change(change):
    global date_filter_col
    date_filter_col = change['new']
    refresh_all(refresh_map_flag=False)
date_col_dropdown.observe(on_date_col_change, names='value')
def on_date_from_change(change):
    global date_filter_from
    date_filter_from = change['new']
    refresh_all(refresh_map_flag=False)
date_from_picker.observe(on_date_from_change, names='value')
def on_date_to_change(change):
    global date_filter_to
    date_filter_to = change['new']
    refresh_all(refresh_map_flag=False)
date_to_picker.observe(on_date_to_change, names='value')
def update_date_picker_visibility(*a):
    show = (date_filter_dropdown.value == 'custom')
    date_from_picker.layout.display = 'block' if show else 'none'
    date_to_picker.layout.display = 'block' if show else 'none'
update_date_picker_visibility()
date_filter_dropdown.observe(update_date_picker_visibility, names='value')

# ---- Filter Row Layout ----
def build_filter_row():
    symbol_area = widgets.Box([symbol_dropdown_outer], layout=widgets.Layout(min_width="125px", max_width="145px", margin="0 14px 0 0"))
    date_area = widgets.Box([date_filter_dropdown], layout=widgets.Layout(min_width="165px", max_width="185px", margin="0 14px 0 0"))
    datecol_area = widgets.Box([date_col_dropdown], layout=widgets.Layout(min_width="165px", max_width="185px", margin="0 14px 0 0"))
    date_range_area = widgets.HBox([date_from_picker, date_to_picker], layout=widgets.Layout(align_items="center", min_width="335px", max_width="355px"))
    row = widgets.HBox(
        [symbol_area, date_area, datecol_area, date_range_area],
        layout=widgets.Layout(width="99%", min_width="800px", align_items="center", flex_wrap="nowrap")
    )
    return row

# ---- Filtering, Sorting, Pagination ----
def filtered_sorted_df():
    df = log_df.copy()
    if selected_symbols:
        df = df[df["symbol"].isin(selected_symbols)]
    if date_filter_mode != 'all':
        col = date_filter_col
        now = datetime.now()
        if date_filter_mode == '1h':
            cutoff = now - timedelta(hours=1)
            df = df[df[col].apply(lambda d: dt_from_str(d) >= cutoff if pd.notnull(d) and d else False)]
        elif date_filter_mode == '24h':
            cutoff = now - timedelta(hours=24)
            df = df[df[col].apply(lambda d: dt_from_str(d) >= cutoff if pd.notnull(d) and d else False)]
        elif date_filter_mode == 'custom':
            try: from_dt = dt_from_str(date_from_picker.value)
            except: from_dt = None
            try: to_dt = dt_from_str(date_to_picker.value)
            except: to_dt = None
            if from_dt is not None:
                df = df[df[col].apply(lambda d: dt_from_str(d) >= from_dt if pd.notnull(d) and d else False)]
            if to_dt is not None:
                df = df[df[col].apply(lambda d: dt_from_str(d) <= to_dt if pd.notnull(d) and d else False)]
    reverse = sort_desc
    col = sort_col
    if col in df.columns:
        df = df.sort_values(col, ascending=not reverse, na_position="last")
    return df.reset_index(drop=True)

def paged_df():
    df = filtered_sorted_df()
    total = len(df)
    start = page_num * page_size
    end = start + page_size
    return df.iloc[start:end], total

# ---- Table Rendering ----
table_box = widgets.Output()
page_size_dropdown = widgets.Dropdown(
    options=[10, 20, 50, 100], value=10, description='Rows/page:'
)
def on_page_size_change(change):
    global page_size, page_num
    page_size = int(change['new'])
    page_num = 0
    refresh_all(refresh_map_flag=False)
page_size_dropdown.observe(on_page_size_change, names='value')

# --- Table CSS: Used for HTML and widget mode (via styles in .layout) ---
table_cell_style = dict(
    padding='7px 9px', min_width='55px', max_width='340px',
    font_size='15px', border='1px solid #333', justify_content='center', align_items='center'
)
table_row_style = widgets.Layout(
    display='flex', flex_flow='row', align_items='center', min_height='38px'
)
table_header_style = dict(**table_cell_style, font_weight='bold', background='#252525')

def cell_layout(**kw):
    """Return a Layout merged with table_cell_style and any overrides."""
    return widgets.Layout(**{**table_cell_style, **kw})

table_css = """
<style>
.nato-log-scrollwrap { overflow-x:auto; background:#151515; padding-bottom:6px; border-radius:7px; }
.nato-log-table { border-collapse:collapse; min-width:1260px; width:max-content; background:#181818; color:#fff; font-family:sans-serif; }
.nato-log-table th, .nato-log-table td { border:1px solid #333; padding:7px 9px; text-align:left; font-size:15px; vertical-align:middle; }
.nato-log-table th { background:#252525; font-weight:bold; border-bottom:2px solid #666; color:#fff; }
.nato-log-table td img { display:block; margin:auto; }
.nato-log-table tr:nth-child(even) { background:#232323; }
.nato-log-table tr:hover { background:#263046; }
</style>
"""

def build_html_table():
    df, total = paged_df()
    n = len(df)
    header = (['Symbol'] + [col.capitalize().replace('_',' ') for col in log_columns[1:]])
    header_html = ''.join(f'<th>{h}</th>' for h in header)
    rows_html = ""
    for i in range(n):
        row = df.iloc[i]
        icon_url = svg_to_datauri(SVGs.get(row['symbol'], SVGs[symbol_options[0]]))
        icon_html = f'<img src="{icon_url}" width="36" height="27"/>'
        tds = [f'<td style="text-align:center;vertical-align:middle;padding:7px 9px;">{icon_html}</td>']
        for col in log_columns[1:]:
            val = row[col]
            val = str(val) if col == "coords" and val is not None else ("" if pd.isnull(val) else val)
            tds.append(f'<td style="vertical-align:middle;padding:7px 9px;">{val}</td>')
        rows_html += '<tr>' + ''.join(tds) + '</tr>'
    table_html = f"""{table_css}
    <div class="nato-log-scrollwrap">
    <table class="nato-log-table">
        <thead><tr>{header_html}</tr></thead>
        <tbody>{rows_html}</tbody>
    </table>
    </div>
    """
    return table_html

def build_widget_table():
    from ipywidgets import GridspecLayout
    df, total = paged_df()
    n = len(df)
    col_names = ['Symbol'] + [col.capitalize().replace('_',' ') for col in log_columns[1:]] + ['Delete', 'Edit']
    ncols = len(col_names)
    # Same column width order as HTML CSS (min/max widths from your .nato-log-table CSS)
    col_widths = [
        '80px',   # Symbol
        '150px',  # When
        '150px',  # Coords
        '150px',  # Reported at
        '150px',  # Reporter id
        '150px',  # Reporter type
        '150px',   # Source reliability
        '150px',   # Info confidence
        '300px',  # Description
        '150px',  # Source reference
        '52px',   # Delete
        '52px',   # Edit
    ]
    header_style = (
        "background:#252525;"
        "font-weight:bold;"
        "color:#fff;"
        "font-size:15px;"
        "text-align:center;"
        "vertical-align:middle;"
        "border:1px solid #333;"
        "padding:7px 9px;"
        "font-family:sans-serif;"
        "min-height:38px; max-height:38px;"
    )
    cell_style = (
        "font-size:15px;"
        "text-align:center;"
        "vertical-align:middle;"
        "border:1px solid #333;"
        "padding:7px 9px;"
        "font-family:sans-serif;"
        "min-height:38px; max-height:38px;"
        "background:#181818;"
    )
    # Build the grid
    grid = GridspecLayout(n + 1, ncols, width='max-content', min_width='1260px')

    # Set column widths exactly as HTML
    col_template = " ".join(col_widths[:ncols])
    grid.layout.grid_template_columns = col_template

    # Fill header
    for j, h in enumerate(col_names):
        grid[0, j] = widgets.HTML(
            f'<div style="{header_style}; text-align:center; display:flex; align-items:center; justify-content:center; width:100%; height:100%">{h}</div>',
            layout=widgets.Layout(
                width=col_widths[j] if j < len(col_widths) else "110px",
                min_width=col_widths[j] if j < len(col_widths) else "110px",
                max_width=col_widths[j] if j < len(col_widths) else "110px",
                min_height="38px", max_height="38px"
            )
        )

    # Fill data
    for i in range(n):
        row = df.iloc[i]
        icon_url = svg_to_datauri(SVGs.get(row['symbol'], SVGs[symbol_options[0]]))
        grid[i+1, 0] = widgets.HTML(
            f'<img src="{icon_url}" width="36" height="27"/>',
            layout=widgets.Layout(width=col_widths[0], min_width=col_widths[0], max_width=col_widths[0], min_height="38px", max_height="38px", justify_content="center", align_items="center")
        )
        col_vals = [
            str(row['when']),
            str(row['coords']),
            str(row['reported_at']),
            str(row['reporter_id']),
            str(row['reporter_type']),
            str(row['source_reliability']),
            str(row['info_confidence']),
            str(row['description']),
            str(row['source_reference'])
        ]
        for j, val in enumerate(col_vals):
            grid[i+1, j+1] = widgets.HTML(
                f'<div style="{cell_style}">{val}</div>',
                layout=widgets.Layout(width=col_widths[j+1], min_width=col_widths[j+1], max_width=col_widths[j+1], min_height="38px", max_height="38px", justify_content="center", align_items="center")
            )
        # Delete/Edit buttons
        del_btn = widgets.Button(description="X", button_style='danger', layout=widgets.Layout(width=col_widths[-2], min_width=col_widths[-2], max_width=col_widths[-2], height="38px", margin="0"))
        edit_btn = widgets.Button(description="✎", button_style='info', layout=widgets.Layout(width=col_widths[-1], min_width=col_widths[-1], max_width=col_widths[-1], height="38px", margin="0"))
        idx = df.index[i]
        def del_row(btn, idx=idx):
            global log_df
            log_df = log_df.drop(idx).reset_index(drop=True)
            refresh_all(refresh_map_flag=False)
        def begin_edit(btn, i=i):
            editing_row_idx[0] = i
            refresh_table()
        del_btn.on_click(del_row)
        edit_btn.on_click(begin_edit)
        grid[i+1, ncols-2] = del_btn
        grid[i+1, ncols-1] = edit_btn
    # Wrap in scrollable box
    scroll = widgets.Box([grid], layout=widgets.Layout(overflow_x='auto', overflow_y='visible', min_width='1260px', width='100%', background='#151515', border_radius='7px', padding='0 0 6px 0'))
    return scroll



def build_table():
    if not edit_mode.value:
        table_html = build_html_table()
        # Pagination for HTML mode
        total_filtered = len(filtered_sorted_df())
        pages = max(1, (total_filtered + page_size - 1) // page_size)
        cur_page = page_num + 1
        page_lbl = widgets.Label(f"Page {cur_page} / {pages} ({total_filtered} rows)", layout=widgets.Layout(margin="0 8px", min_width="160px"))
        prev_btn = widgets.Button(description="←", layout=widgets.Layout(width='30px', height='32px'))
        next_btn = widgets.Button(description="→", layout=widgets.Layout(width='30px', height='32px'))
        def goto_prev(btn): 
            global page_num
            page_num = max(0, page_num-1)
            refresh_all(refresh_map_flag=False)
        def goto_next(btn): 
            global page_num
            page_num = min(pages-1, page_num+1)
            refresh_all(refresh_map_flag=False)
        prev_btn.on_click(goto_prev)
        next_btn.on_click(goto_next)
        pag = widgets.HBox([prev_btn, page_lbl, next_btn, page_size_dropdown], layout=widgets.Layout(justify_content="flex-start",margin="6px 0"))
        return widgets.VBox([widgets.HTML(value=table_html), pag])
    else:
        return build_widget_table()

edit_mode.observe(lambda c: refresh_table(), names='value')

# ---- Map logic ----
m = Map(center=(51.5, -0.12), zoom=12, layout=widgets.Layout(width="98%", height="390px"))
marker_objects = []
def refresh_map():
    for mk, idx in marker_objects:
        try: m.remove_layer(mk)
        except: pass
    marker_objects.clear()
    size = icon_size_for_zoom(m.zoom)
    for idx, row in filtered_sorted_df().iterrows():
        icon = Icon(icon_url=svg_to_datauri(SVGs.get(row["symbol"], SVGs[symbol_options[0]])), icon_size=size, icon_anchor=[size[0]//2, size[1]//2])
        marker = Marker(location=row["coords"], icon=icon)
        m.add_layer(marker)
        marker_objects.append((marker, idx))

# ---- Manual Map Update Button ----
update_map_btn = widgets.Button(
    description="Update Map", button_style='primary', icon='refresh',
    layout=widgets.Layout(width="130px", min_height="38px", max_height="38px", margin='0 0 0 14px')
)
def update_map_btn_clicked(btn):
    refresh_map()
update_map_btn.on_click(update_map_btn_clicked)

# ---- Create Button ----
create_btn = widgets.Button(
    description="Create", button_style='success', icon='plus',
    layout=widgets.Layout(width="110px", min_height="38px", max_height="38px", margin='0 0 0 16px')
)
def create_btn_clicked(btn):
    creating_row[0] = True
    editing_row_idx[0] = None
    refresh_table()
create_btn.on_click(create_btn_clicked)

# ---- Main refresh ----
def refresh_all(refresh_map_flag=True):
    filter_row.children = [build_filter_row()]
    refresh_table()
    if refresh_map_flag:
        refresh_map()

def refresh_table():
    table_box.clear_output(wait=True)
    with table_box: display(build_table())

def handle_click(**kwargs):
    if kwargs.get('type') == 'click':
        latlng = kwargs['coordinates']
        current_time = now_str()
        new_row = {
            "symbol": symbol_dropdown.value,
            "when": current_time,
            "coords": list(latlng),
            "reported_at": current_time,
            "reporter_id": "", "reporter_type": "",
            "source_reliability": "", "info_confidence": "",
            "description": "", "source_reference": ""
        }
        global log_df
        log_df = pd.concat([log_df, pd.DataFrame([new_row])], ignore_index=True)
        refresh_all(refresh_map_flag=True)
m.on_interaction(handle_click)
def on_zoom_change(event): refresh_map()
m.observe(on_zoom_change, names='zoom')

filter_row = widgets.HBox([])
main_box = widgets.VBox([
    widgets.HBox([symbol_dropdown, add_info]),
    m,
    widgets.HBox(
        [filter_row, update_map_btn, create_btn, edit_mode],
        layout=widgets.Layout(align_items="center", min_height="38px", max_height="38px")
    ),
    table_box
], layout=widgets.Layout(width="99%"))
refresh_all(refresh_map_flag=True)
display(main_box)


In [ ]:
print("hello")